<a href="https://colab.research.google.com/github/lenocahya/ir_wisata_semarang/blob/main/crawl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Crawling


In [1]:
import re
import string
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

**Fungsi Crawling**

In [2]:
def crawl(url, num_page):
    
    result = []
    req = requests.get(url)
    soup = BeautifulSoup(req.text, "lxml")
    
    # Membuka paging page 
    paging = soup.find_all("nav",{'class':'pagination'})
    paging_link = paging[0].find_all('a',{'class':'pagination__page'})
    last_page = num_page

    # looping untuk membuka setip halaman
    for i in range(1,last_page+1):
        print(url+'page/'+str(i))

        # mencari daftar link dari artiker yang ada pada setiap halaman
        req = requests.get(url+'page/'+str(i))
        soup = BeautifulSoup(req.text, "lxml")
        news_links = soup.find_all("div",{'class':'thumb-container thumb-65'})
        # print(news_links)

        # looping membuka artikel setiap halaman
    
        for idx,news in enumerate(news_links):
            news_dict = {}

            #find news title
            title_news= news.find('h1',{'class':'entry__title'}).text.replace("\n", "").title()
    

            #find urll news
            url_news = news.find('a',{'class':''}).get('href')

            
            #find news content in url
            req_news =  requests.get(url_news)
            soup_news = BeautifulSoup(req_news.text, "lxml")

            #find news content 
            news_content = soup_news.find("div",{'class':'entry__article-wrap'})

            #find paragraph in news content 
            p = news_content.find_all('p')
            content = ' '.join(item .text for item in p)
            news_content = content.encode('utf8','replace')

            #wrap in dictionary 
            news_dict['no']=idx+1
            news_dict['url'] = url_news
            news_dict['title'] = title_news
            news_dict['content'] = news_content
            result.append(news_dict)
         
    return result

In [3]:
def crawl_kuliner(url):
    # mencari daftar link dari artikel yang ada pada setiap halaman
    result = []
    req = requests.get(url)
    soup = BeautifulSoup(req.text, "lxml")  
    news_links = soup.find_all("div",{'class':'thumb-container thumb-65'})

    # looping membuka artikel setiap halaman

    for idx,news in enumerate(news_links):
        news_dict = {}

        #find news title
        title_news= news.find('h1',{'class':'entry__title'}).text.replace("\n", "").title()


        #find urll news
        url_news = news.find('a',{'class':''}).get('href')

        
        #find news content in url
        req_news =  requests.get(url_news)
        soup_news = BeautifulSoup(req_news.text, "lxml")

        #find news content 
        news_content = soup_news.find("div",{'class':'entry__article-wrap'})

        #find paragraph in news content 
        p = news_content.find_all('p')
        content = ' '.join(item .text for item in p)
        news_content = content.encode('utf8','replace')

        #wrap in dictionary 
        news_dict['no']=idx+1
        news_dict['url'] = url_news
        news_dict['title'] = title_news
        news_dict['content'] = news_content
        result.append(news_dict)
         
    return result

In [4]:
url = 'https://visitjawatengah.jatengprov.go.id/id/filter/kabupaten-semarang/result/kuliner'
kuliner1 = crawl_kuliner(url)
kuliner1 = pd.DataFrame(kuliner1)

In [5]:
url = 'https://visitjawatengah.jatengprov.go.id/id/filter/kota-semarang/result/kuliner'
kuliner2 = crawl_kuliner(url)
kuliner2 = pd.DataFrame(kuliner2)

In [6]:
kuliner = kuliner1.append(kuliner2)

In [7]:
url = 'https://visitjawatengah.jatengprov.go.id/id/regency/kabupaten-semarang/destinasi-wisata/'
wisata1 = crawl(url, 4)
wisata1 = pd.DataFrame(wisata1)

https://visitjawatengah.jatengprov.go.id/id/regency/kabupaten-semarang/destinasi-wisata/page/1
https://visitjawatengah.jatengprov.go.id/id/regency/kabupaten-semarang/destinasi-wisata/page/2
https://visitjawatengah.jatengprov.go.id/id/regency/kabupaten-semarang/destinasi-wisata/page/3
https://visitjawatengah.jatengprov.go.id/id/regency/kabupaten-semarang/destinasi-wisata/page/4


In [8]:
url = 'https://visitjawatengah.jatengprov.go.id/id/regency/kota-semarang/destinasi-wisata/'
wisata2 = crawl(url, 3)
wisata2 = pd.DataFrame(wisata2)

https://visitjawatengah.jatengprov.go.id/id/regency/kota-semarang/destinasi-wisata/page/1
https://visitjawatengah.jatengprov.go.id/id/regency/kota-semarang/destinasi-wisata/page/2
https://visitjawatengah.jatengprov.go.id/id/regency/kota-semarang/destinasi-wisata/page/3


In [9]:
wisata = wisata1.append(wisata2)

In [10]:
crwl = wisata.append(kuliner)

In [11]:
crwl.head(None)

,no,url,title,content
0,1,https://visitjawatengah.jatengprov.go.id/id/re...,Puri Kayana Camping Ground,b'#SobatWisata\xc2\xa0kangen ngecamp nggak nih...
1,2,https://visitjawatengah.jatengprov.go.id/id/re...,Mengunjungi Wisata Religi Nyatnyono,b'Desa Nyatnyono di lereng Gunung Ungaran dike...
2,3,https://visitjawatengah.jatengprov.go.id/id/re...,"Candi Dukuh, Petilasan Prabu Brawijaya","b'Candi Dukuh terletak di Desa Rowoboni, Kecam..."
3,4,https://visitjawatengah.jatengprov.go.id/id/re...,5 Hal Spesial Di Desa Wisata Lerep,b'Jalan-jalan ke Semarang? Mampirlah ke Desa W...
4,5,https://visitjawatengah.jatengprov.go.id/id/re...,All In One In Sunrise Hill Gedong Songo,b'#SobatWisata\xc2\xa0lagi cari referensi rest...
...,...,...,...,...
4,5,https://visitjawatengah.jatengprov.go.id/id/re...,Mi Kopyok,b'adalah jenis makanan yang biasanya dijual di...
5,6,https://visitjawatengah.jatengprov.go.id/id/re...,Tahu Gimbal,b'Racikan tahu gimbal terdiri dari potongan ta...
6,7,https://visitjawatengah.jatengprov.go.id/id/re...,Lumpia,b'Kudapan yang terbuat dari rebung dibungkus d...
7,8,https://visitjawatengah.jatengprov.go.id/id/re...,Wedang Tahu,b'Wedang Tahu adalah sejenis minuman yang bera...


In [12]:
crwl.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 87 entries, 0 to 8
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   no       87 non-null     int64 
 1   url      87 non-null     object
 2   title    87 non-null     object
 3   content  87 non-null     object
dtypes: int64(1), object(3)
memory usage: 3.4+ KB


**Cleaning Dokumen hasil crwaling**

In [13]:
def documents_cleaning(doc):
  docs_clean = []

  for d in doc:
      html = d.decode("utf-8")                              # Men-decode hasil scrapy dari web yang masih terenkripsi utf-8 dari html
      document_test = re.sub(r'[^\x00-\x7F]+', ' ', html)
      document_test = re.sub(r'#\w+', '', document_test)
      docs_clean.append(document_test)

  return docs_clean

In [14]:
# menghilangkan simbol-simbol, tag, url, dan huruf atau karakter yang tidak digunakan
def case_folding(doc):      
      doc = re.sub(r'@\w+', '', doc)
      doc = re.sub("http.+", " ", doc) 
      doc = re.sub("www.+", " ", doc)
      doc = re.sub(".+.com$", " ", doc)
      doc = doc.lower()
      doc = re.sub(r'[%s]' % re.escape(string.punctuation), ' ', doc)
      doc = re.sub(r'[0-9]', '', doc)
      doc = re.sub(r"\b[a-zA-Z]\b", "",doc)
      doc = re.sub(r'\s{2,}', ' ', doc)
      doc = doc.replace("html", " ")
      doc = re.sub(r'\s+', ' ', doc)    
      return doc
      

In [15]:
crwl['content'] = documents_cleaning(crwl['content'])

In [16]:
crwl['clean_content'] = (crwl['content']).apply(case_folding)

In [17]:
crwl.head()

,no,url,title,content,clean_content
0,1,https://visitjawatengah.jatengprov.go.id/id/re...,Puri Kayana Camping Ground,kangen ngecamp nggak nih? Cuss berkemah denga...,kangen ngecamp nggak nih cuss berkemah dengan...
1,2,https://visitjawatengah.jatengprov.go.id/id/re...,Mengunjungi Wisata Religi Nyatnyono,Desa Nyatnyono di lereng Gunung Ungaran dikena...,desa nyatnyono di lereng gunung ungaran dikena...
2,3,https://visitjawatengah.jatengprov.go.id/id/re...,"Candi Dukuh, Petilasan Prabu Brawijaya","Candi Dukuh terletak di Desa Rowoboni, Kecamat...",candi dukuh terletak di desa rowoboni kecamata...
3,4,https://visitjawatengah.jatengprov.go.id/id/re...,5 Hal Spesial Di Desa Wisata Lerep,Jalan-jalan ke Semarang? Mampirlah ke Desa Wis...,jalan jalan ke semarang mampirlah ke desa wisa...
4,5,https://visitjawatengah.jatengprov.go.id/id/re...,All In One In Sunrise Hill Gedong Songo,lagi cari referensi resto? Penginapan? Spot s...,lagi cari referensi resto penginapan spot sel...


In [18]:
hasil = crwl[['no','title','content','clean_content','url']]
hasil.to_csv('crawl_wisata.csv', index=False)
hasil.head()

,no,title,content,clean_content,url
0,1,Puri Kayana Camping Ground,kangen ngecamp nggak nih? Cuss berkemah denga...,kangen ngecamp nggak nih cuss berkemah dengan...,https://visitjawatengah.jatengprov.go.id/id/re...
1,2,Mengunjungi Wisata Religi Nyatnyono,Desa Nyatnyono di lereng Gunung Ungaran dikena...,desa nyatnyono di lereng gunung ungaran dikena...,https://visitjawatengah.jatengprov.go.id/id/re...
2,3,"Candi Dukuh, Petilasan Prabu Brawijaya","Candi Dukuh terletak di Desa Rowoboni, Kecamat...",candi dukuh terletak di desa rowoboni kecamata...,https://visitjawatengah.jatengprov.go.id/id/re...
3,4,5 Hal Spesial Di Desa Wisata Lerep,Jalan-jalan ke Semarang? Mampirlah ke Desa Wis...,jalan jalan ke semarang mampirlah ke desa wisa...,https://visitjawatengah.jatengprov.go.id/id/re...
4,5,All In One In Sunrise Hill Gedong Songo,lagi cari referensi resto? Penginapan? Spot s...,lagi cari referensi resto penginapan spot sel...,https://visitjawatengah.jatengprov.go.id/id/re...
